In [ ]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
#All the libraries that we need
import string
import nltk
from collections import Counter
import unicodedata
import re

from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import time

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import numpy as np 
import pandas as pd
import os
import nltk
import gensim
from string import punctuation
from os import listdir
from collections import Counter

#Common functions
punc=['…','،','۔','،','۔','؟' , '.', ',']
NFC = [('آ', 'آ'), ('ة', 'ۃ'), ('ـ', ''), ('ك', 'ک'), ('ه', 'ہ'), ('ۀ', 'ۂ'), 
       ('ى', 'ی'), ('ي', 'ی'), ('٠', '۰'), ('١', '۱'), ('٢', '۲'), 
       ('٣', '۳'), ('٤', '۴'), ('٥', '۵'), ('٦', '۶'), ('٧', '۷'), ('٨', '۸'), 
       ('٩', '۹'), ('ۓ', 'ئے'), ('ۓ', 'ئے'), ('ئ', 'ئی'),
       ("ِ", ""), ("ِ", ""),("ُ", ""),("َ", ""),("ْ", ""),("ٰ", ""),("ً", ""),("ّ", "")      
      ]

# load file into memory
def load_file(filename):
    file = open(filename, 'r', encoding='utf8')
    text = file.read()
    file.close()
    return text

def read_as_dict_for_lemma(file):
    text = load_file(file)
    lines = text.split("\n")
    wd = dict()
    for line in lines:
        toks = line.split(":")
        if len(toks)==0 or len(toks)==1:
            continue
        wd[toks[0].strip()] = toks[1].strip()
    return wd

#Loading Lemmatizer
dictionary = read_as_dict_for_lemma('Humayoun_lemmatizer.txt')

def lemma(word, dictionary):
    if word in dictionary:
        return dictionary[word]
    else:
        return word

def read_stopwords(file):
    text = load_file(file)
    tokens = text.split()
    tokens = set(tokens)
    return tokens

def mkNFC(token):
    for (ch1,ch2) in NFC:
        token.replace(ch1,ch2)
    return token

# turn a txt into clean tokens
def mk_tokens_vocab(txt, stw=None, lma=None, ngram=1, how="all", ch_ngram=2, ch_how=None):
    # split into tokens by white space
    tokens = txt.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation+"".join(punc))
    tokens = [w.translate(table) for w in tokens]
    tokens = [mkNFC(token) for token in tokens]
    if stw!=None:
        # filter out stop words
        stop_words = set(read_stopwords('stopword_Urmono.txt'))
        tokens = [w for w in tokens if not w in stop_words]
        # filter out short tokens
        tokens = [word for word in tokens if len(word) > 1]
        tokens = [word for word in tokens if word!='کرنا']
        tokens = [word for word in tokens if word!='پرنا']
        tokens = [word for word in tokens if word!='رہنا']
        tokens = [word for word in tokens if word!='جانا']
        tokens = [word for word in tokens if word!='کَیا']
    if lma!= None:
        #apply lemmatization
        tokens = [lemma(token, dictionary) for token in tokens]
    
    
    #getting the char ngrams
    charngrams=[]
    if ch_how!=None:
        charngrams = mkCNgram(tokens, ch_ngram, ch_how)
    
    wordngrams = mkNgram(tokens, ngram, how)
    tokens = charngrams + wordngrams
    
    #print(tokens)
    return tokens

# turn a txt into clean tokens
def mk_tokens(txt, vocab, stw=None, lma=None, ngram=1, how="all", ch_ngram=2, ch_how=None):
    tokens = mk_tokens_vocab(txt, stw, lma, ngram, how, ch_ngram, ch_how)
    tokens = [t for t in tokens if t in vocab]
    return tokens

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab, stw=None, lma=None, 
                     ngram=1, how="all", ch_ngram=2, ch_how=None):
    # load file
    text = load_file(filename)
    # clean doc
    tokens = mk_tokens_vocab(text, stw, lma, ngram, how, ch_ngram, ch_how)
    vocab.update(tokens)


def extract_text_label_from_dict(table):
    txts=list()
    labels=list()
    for item in table:
        (txt, clabel) = table[item]
        txts.append(txt)
        labels.append(clabel)
    return txts, labels

def extract_ids_from_dict(table):
    return list(table.keys())

def mkNgram(tokens, n=1, how="all"):
    if how=="all":
        grams = list()
        for r in range(n):
            r+=1
            grams.extend(["_".join( tokens[i:i+r] ) for i in range(len(tokens)-r+1)]) 
        return grams
    else:
        return ["_".join( tokens[i:i+n] ) for i in range(len(tokens)-n+1)]


#char ngrams
def mkCNgram(tokens, L, cn_how="all"):
    tokens = "".join( tokens[:] )
    tokens = mkNgram(tokens, L, cn_how)
    return tokens


# load all docs in a directory
def process_vocab_docs(directory, vocab, stw=None, lma=None, 
                     ngram=1, how="all", ch_ngram=2, ch_how=None):
    # walk through all files in the folder
    for filename in listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab, stw, lma, 
                         ngram, how, ch_ngram, ch_how)
        
#=load_file('Urdu-Fake-news-detection-FIRE2021-main/Train/Train/Fake/bs1.txt')       
#t

# load all docs in a directory
def process_docs(directory, vocab, 
                     stw=None, lma=None, 
                     ngram=1, how="all", 
                     ch_ngram=2, ch_how=None):
    documents = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        doc = load_file(path)
        # clean doc
        tokens = mk_tokens(doc, vocab, 
                           stw, lma, 
                           ngram, how, 
                           ch_ngram, ch_how)
        # add to list
        documents.append(tokens)
    return documents

# load all docs in a directory
def process_docs_test(directory, vocab, 
                     stw=None, lma=None, 
                     ngram=1, how="all", 
                     ch_ngram=2, ch_how=None):
    documents = list()
    files=list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        doc = load_file(path)
        # clean doc
        tokens = mk_tokens(doc, vocab, 
                           stw, lma, 
                           ngram, how, 
                           ch_ngram, ch_how)
        # add to list
        documents.append(tokens)
        files.append(filename.split(".")[0])
    return (documents, files)

# Deep Convolutional Neural Network for Sentiment Analysis

In [ ]:
from numpy import array
import numpy
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd
from sklearn.metrics import precision_score, recall_score,accuracy_score, f1_score, confusion_matrix

# fit a tokenizer
def create_tokenizer(lines, v_char_level):
    tokenizer = Tokenizer(char_level=v_char_level)
    tokenizer.fit_on_texts(lines)
    return tokenizer

# calculate the maximum document length
def max_length(lines):
    return max([len(s) for s in lines])


def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length,  padding='post')
    return padded

# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(32, 1, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D()(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(32, 2, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D()(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(32, 3, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D()(drop3)
    flat3 = Flatten()(pool3)
    # channel 4
    inputs4 = Input(shape=(length,))
    embedding4 = Embedding(vocab_size, 100)(inputs4)
    conv4 = Conv1D(32, 4, activation='relu')(embedding4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling1D()(drop4)
    flat4 = Flatten()(pool4)
    # channel 5
    inputs5 = Input(shape=(length,))
    embedding5 = Embedding(vocab_size, 100)(inputs5)
    conv5 = Conv1D(32, 5, activation='relu')(embedding5)
    drop5 = Dropout(0.5)(conv5)
    pool5 = MaxPooling1D()(drop5)
    flat5 = Flatten()(pool5)
    # channel 6
    inputs6 = Input(shape=(length,))
    embedding6 = Embedding(vocab_size, 100)(inputs6)
    conv6 = Conv1D(32, 6, activation='relu')(embedding6)
    drop6 = Dropout(0.5)(conv6)
    pool6 = MaxPooling1D()(drop6)
    flat6 = Flatten()(pool6)
    
    # merge
    merged = concatenate([flat1, flat2, flat3, flat4, flat5, flat6])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4, inputs5, inputs6], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    #model.summary()
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

def mkExp():
    # define vocab  
    vocab = Counter()
    # add all text to vocab
    process_vocab_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Train/Fake', vocab,
                       stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    process_vocab_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Train/Real', vocab,
                       stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    # add all text to vocab
    process_vocab_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Test/Fake', vocab,
                       stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    process_vocab_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Test/Real', vocab,
                       stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)


    print("Most common:", vocab.most_common(5))
    vocab = set(vocab)
    print("Vocabulary Size:", len(vocab))


    # load all training reviews
    fake_docs = process_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Train/Fake', vocab,
                                stw=v_stw, lma=v_lma, 
                               ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    real_docs = process_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Train/Real', vocab,
                                stw=v_stw, lma=v_lma, 
                               ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    train_docs = real_docs + fake_docs
    y = ([0 for _ in range(len(real_docs))] + [1 for _ in range(len(fake_docs))])

    #R->0
    #F->1
    #{0: 750, 1: 550})


    # load all test reviews
    fake_test_docs = process_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Test/Fake', vocab,
                                stw=v_stw, lma=v_lma, 
                               ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    real_test_docs = process_docs('Urdu-Fake-news-detection-FIRE2021-main/Train/Test/Real', vocab,
                                stw=v_stw, lma=v_lma, 
                               ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
    test_docs = real_test_docs + fake_test_docs
    ytest = ([0 for _ in range(len(real_test_docs))] + [1 for _ in range(len(fake_test_docs))])

    docs = train_docs+test_docs
    y = y+ytest
    y=array(y)
    # summarize the new class distribution
    counter = Counter(y)
    print(counter)


    #Read the test directory   
    test_x, files = process_docs_test('Urdu-Fake-news-detection-FIRE2021-main/Test', vocab,
                               stw=v_stw, lma=v_lma, 
                               ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)

    ## create tokenizer
    tokenizer = create_tokenizer(docs, v_char_level) 
    # calculate max document length
    length = max_length(docs)
    ## load all training reviews
    X = encode_text(tokenizer, docs, length)


    # calculate max document length
    length = max_length(X)
    print('Max document length: %d' % length)

    # calculate vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
    print('Vocabulary size: %d' % vocab_size)
    # encode data

    # encode training data set
    Xtest = encode_text(tokenizer, test_x, length)


    # define model
    model = define_model(length, vocab_size)
    # fit model
    model.fit([X,X,X,X,X,X], y, epochs=10, batch_size=7)
    print("Model trained....")
    
    model.save(result_file+'.h5')
    # make probability predictions with the model
    predictions_scores = model.predict([Xtest,Xtest,Xtest,Xtest,Xtest,Xtest])

    test_pred_labels = list()
    for score in predictions_scores:
        if score>=0.5:
            test_pred_labels.append( 1 )
        else:
            test_pred_labels.append( 0 )

    text='File_No,Real/Fake\n'
    for i in range(len(files)):
        if test_pred_labels[i]==1:
            text+=files[i]+",F\n"
        else:
            text+=files[i]+",R\n"

    afile = open(result_file,"w", encoding="utf8")
    afile.write(text)
    afile.close()
    
    orig=pd.read_excel('Orignal_labels.xlsx')
    #orig.head()
    # Added a new column to make labels in Numeric form, like F --> 1, R --> 0
    orig['label'] = orig['Real/Fake'].apply(lambda x: 1 if x == 'F' else 0)
    
    pred=pd.read_csv(result_file)
    #pred.head()
    
    # Added a new column to make labels in Numeric form, like F --> 1, R --> 0
    pred['label'] = pred['Real/Fake'].apply(lambda x: 1 if x == 'F' else 0)

    pred= pred.sort_values(by='File_No')
    cm = confusion_matrix(orig['label'], pred['label'])
    tn, fp, fn, tp = cm.ravel()
    #######################################################
    ###################For Fake Class######################

    prec_fake = tp/(tp + fp)
    print('Precision for Fake class :', prec_fake)

    rec_fake = tp/(tp + fn)
    print('Recall for Fake class :', rec_fake)

    f1_fake = 2 * prec_fake * rec_fake / ( prec_fake + rec_fake)
    print('F1_Fake :', f1_fake)

    #######################################################
    ###################For Real Class######################
    prec_real = tn/(tn + fn)
    print('\nPrecision for Real class :', prec_real)

    rec_real = tn/(tn + fp)
    print('Recall for Real class :', rec_real)

    f1_real = 2 * prec_real * rec_real / ( prec_real + rec_real)
    print('F1_Real :', f1_real)

    #################################################################
    f1_mac = (f1_real + f1_fake )/2
    print('\nF1_Macro :', f1_mac)
    print('The Accuracy score is: ', accuracy_score(orig['label'], pred['label']))


    #Calculate metrics (f1 ) for each label, 
    #and find their average weighted by support (the number of true instances for each label).
    #This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

    f1_weighted = (250 /400) * f1_real + (150 /400) * f1_fake
    print('\nF1_Average :', f1_weighted)
    print("-------------------------------------------------------")


In [ ]:
# 6 layers
v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 0
v_ch_how = None #"exact"

v_char_level=False # word level tokenizer
print("==============================")
for i in range(1,4):
    result_file="WordLevel-CNN-"+str(v_stw)+"-"+str(v_lma)+"-"+str(v_ngram)+"-"+str(v_ch_ngram)+"ch-"+str(v_ch_how)+"-Embedding-100-Conv1D-32-6Layers-iter"+str(i)+".csv"
    mkExp()
print("==============================")

In [ ]:
# 6 layers
v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 0
v_ch_how = None #"exact"

v_char_level=True # word level tokenizer
print("==============================")
for i in range(1,4):
    result_file="CharLevel-CNN-"+str(v_stw)+"-"+str(v_lma)+"-"+str(v_ngram)+"-"+str(v_ch_ngram)+"ch-"+str(v_ch_how)+"-Embedding-100-Conv1D-32-6Layers-iter"+str(i)+".csv"
    mkExp()
print("==============================")

In [ ]:
# 6 layers
v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 2
v_ch_how = "exact" #"exact"

v_char_level=True # word level tokenizer
print("==============================")
for i in range(1,4):
    result_file="CharLevel-CNN-"+str(v_stw)+"-"+str(v_lma)+"-"+str(v_ngram)+"-"+str(v_ch_ngram)+"ch-"+str(v_ch_how)+"-Embedding-100-Conv1D-32-6Layers-iter"+str(i)+".csv"
    mkExp()
print("==============================")

In [ ]:
# 6 layers
v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 3
v_ch_how = "exact" #"exact"

v_char_level=True # word level tokenizer
print("==============================")
for i in range(1,4):
    result_file="CharLevel-CNN-"+str(v_stw)+"-"+str(v_lma)+"-"+str(v_ngram)+"-"+str(v_ch_ngram)+"ch-"+str(v_ch_how)+"-Embedding-100-Conv1D-32-6Layers-iter"+str(i)+".csv"
    mkExp()
print("==============================")

In [ ]:
# 6 layers
v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 3
v_ch_how = "all" #"exact"

v_char_level=True # word level tokenizer
print("==============================")
for i in range(1,4):
    result_file="CharLevel-CNN-"+str(v_stw)+"-"+str(v_lma)+"-"+str(v_ngram)+"-"+str(v_ch_ngram)+"ch-"+str(v_ch_how)+"-Embedding-100-Conv1D-32-6Layers-iter"+str(i)+".csv"
    mkExp()
print("==============================")

In [ ]:
#again word level
# not going to work
# 6 layers
v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 2
v_ch_how = "exact" #"exact"

v_char_level=False # word level tokenizer
print("==============================")
for i in range(1,4):
    result_file="WordLevel-CNN-"+str(v_stw)+"-"+str(v_lma)+"-"+str(v_ngram)+"-"+str(v_ch_ngram)+"ch-"+str(v_ch_how)+"-Embedding-100-Conv1D-32-6Layers-iter"+str(i)+".csv"
    mkExp()
print("==============================")